In [1]:
from denovo_utils.parsers.converters import SpectralisParser
import pandas as pd
import os

In [6]:
root_data="/home/samva/Doctorate/data_directory/denovo_project"
mgf_path=os.path.join(root_data, "mgf_filtered/F06.mgf")
results_dir=os.path.join(root_data, "denovo_results")

In [7]:
parser_spectralis = SpectralisParser(
    mgf_path=mgf_path,
    results_dir=results_dir
)
parser_spectralis.parse(
    path_spectralis=os.path.join(
        results_dir,
        "refinement/spectralis/F06_annotated_rescoring.csv"
    )
)

100%|██████████| 40545/40545 [00:00<00:00, 156640.11it/s]


In [10]:
parser_spectralis.add_xtandem(
    result_path=os.path.join(root_data, "search_results/XTandem_percolator/F06.pout")
)

In [15]:
df = parser_spectralis.dataframe

In [18]:
df.source.value_counts()

source
PepNet           83186
Casanovo4.2.0    80842
InstaNovo        80759
percolator       66592
ContraNovo       40545
Name: count, dtype: int64

In [ ]:
df = df[df]